In [2]:
from helpers_given import load_csv_data
from preprocess import *
from helpers import *

Load data with helpers_given

In [3]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("dataset/", sub_sample=False)

y_train loaded
x_train loaded
x_test loaded


In [5]:
nb_col = x_train.shape[1]
nb_rows = x_train.shape[0]
print(nb_col)
print(nb_rows)

321
328135


Preprocessing of run.py

In [9]:
nb_nans = np.zeros(nb_col)
for i, col in enumerate(x_train.T):
    nb_nans[i] = np.count_nonzero(np.isnan(col))

reduced_data, reduced_features = removing_features(nb_nans, train_ids, x_train)
"""feature_correlation_dict gives an index error

# Get feature correlation dictionary
feature_correlation_dict = create_dictionary_from_correlation(reduced_data,reduced_features,0.6)

max_corrr_feature_dict = {}

# Find the 50 most correlated features
for key, val in feature_correlation_dict.items():
    max_corrr_feature_dict[key] = len(val)

# Sort the dictionary by value in descending order
max_corrr_feature_dict = {k: v for k, v in sorted(max_corrr_feature_dict.items(), key=lambda item: item[1], reverse=True)}

features_to_drop = []
for key in max_corrr_feature_dict.keys():
    features_to_drop.append(key)
    if len(features_to_drop) == 30:
        break

# Define features to keep
features_to_keep = []
for feature in reduced_features:
    if feature not in features_to_drop:
        features_to_keep.append(feature)

# Also replace some features with their calculated counterparts
origin_calculated_features = {
    'WEIGHT2' : 'WTKG3',
    'HEIGHT3' : 'HTM4',
    'ALCDAY5' : '_DRNKWEK',
    'FRUITJU1' : 'FTJUDA1_',
    'FRUIT1' : 'FRUTDA1_',
    'FVBEANS' : 'BEANDAY_',
    'FVGREEN' : 'GRENDAY_',
    'FVORANG' : 'ORNGDAY_',
    'VEGETAB1' : 'VEGEDA1_',
    'STRENGTH' : 'STRFREQ_'
}

# In features_to_keep replace the key of origin_calculated_features with the value
for key, val in origin_calculated_features.items():
    for i, feature in enumerate(features_to_keep):
        if key == feature:
            features_to_keep[i] = val

# Drop duplicates
features_to_keep = list(set(features_to_keep))

# Get the indices of the selected features
selected_features_indices = []
for feature in features_to_keep:
    selected_features_indices.append(reduced_features.index(feature))

selected_features_indices = sorted(selected_features_indices)

# Create a new dataset with keeping the features that are in the selected_features_indices
reduced_data = reduced_data[:,selected_features_indices]

# Also remove the features from the reduced_features list
reduced_features_2 = []
for feature in reduced_features:
    if feature in features_to_keep:
        reduced_features_2.append(feature)

# Remove redundant features
redundant_features = [ 'FMONTH','IDATE','IMONTH','IDAY','IYEAR', 'SEQNO', '_STATE', '_PSU', ]
# Get the indices of these features
redundant_features_indices = []
for feature in redundant_features:
    redundant_features_indices.append(reduced_features_2.index(feature))

# Create a new dataset with removing the features that are in the selected_features_indices
reduced_data = np.delete(reduced_data, redundant_features_indices, 1)
reduced_features_2 = [reduced_features_2[i] for i in range(len(reduced_features_2)) if i not in redundant_features_indices]

# Replace nine values with NaNs
replace_nine_with_nan(reduced_data)

# Replace seven values with NaNs
replace_seven_with_nan(reduced_data)

# Replace 99 values with NaNs
replace_99_with_nan(reduced_data)

# For the _DRNKWEK feature, replace 9990 with NaN
for i in range(reduced_data.shape[0]):
    if reduced_data[i, reduced_features_2.index('_DRNKWEK')] == 9990:
        reduced_data[i, reduced_features_2.index('_DRNKWEK')] = np.nan
"""

# Remove outliers
reduced_data = clean_outliers_modified(reduced_data)

# Replace NaNs with medians
reduced_mean = replace_NaN(reduced_data, method='mean')

# Standardize the data
standardized_x = standardize_data(reduced_mean)

#### Run the methods

In [10]:
from implementations import *
NB_COL = standardized_x.shape[1] # corresponds to 'D' = number of features
NB_ROWS = standardized_x.shape[0] # corresponds to 'N' = number of observations/respondents

In [11]:
# GD
w_mse_gd, loss_mse_gd = mean_squared_error_gd(y_train, standardized_x, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.001)
print(loss_mse_gd)
y_mse_gd = standardized_x @ w_mse_gd
y_mse_gd = convert_predict(y_mse_gd)
print(y_mse_gd)

0.4911846554733139
[0. 1. 1. ... 1. 0. 0.]


In [14]:
# Least squares
w_ls, loss_ls = least_squares(y_train, standardized_x)
print(loss_ls)
y_ls = convert_predict(standardized_x @ w_ls)
print(y_ls)

0.47718365272642943
[0. 1. 0. ... 1. 0. 0.]


In [20]:
# Ridge regression
w_rr, loss_rr = ridge_regression(y_train, standardized_x, lambda_ = 0.5)
print(loss_rr)
y_rr = convert_predict(standardized_x @ w_rr)
print(y_rr)

0.4796769756053548
[0. 1. 0. ... 1. 0. 0.]


In [30]:
# logistic regression (GD)
w_lrgd, loss_lrgd = logistic_regression(y_train, standardized_x, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.2)
print(loss_lrgd)
y_lrgd = convert_predict(standardized_x @ w_lrgd)
#y_lrgd = convert_0_to_minus1(y_sigmoid)
print(y_lrgd)
#log sgd
#w_lrsgd, loss_lrsgd = logistic_regression_SGD(y_train, x_train, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.1, lambda_ = 0.5)


0.5978324012940669
[0. 1. 0. ... 1. 0. 0.]


In [33]:
w_reg, loss_reg = reg_logistic_regression(y_train, standardized_x, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.1, lambda_ = 0.5)
print(loss_reg)
y_reg = convert_predict(standardized_x @ w_reg)
print(y_reg)
# reg log sgd
#w_reg_lrsgd, loss_reg_lrsgd = reg_logistic_regression_SGD(y_train, x_train, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.1, lambda_ = 0.5)

0.6261911346748003
[0. 1. 0. ... 1. 0. 0.]


### OLD VERSION:

Take care of the NaNs

In [ ]:
nb_nans = np.zeros(nb_col)
for i, col in enumerate(x_train.T):
    nb_nans[i] = np.count_nonzero(np.isnan(col))
plt.figure(figsize=(10, 5))
plt.hist(nb_nans, bins=20)
plt.title("Number of NaNs per feature before")


In [ ]:
reduced_x_train, reduced_ids = removing_features(nb_nans, train_ids, x_train)

for i, col in enumerate(reduced_x_train.T):
    nb_nans[i] = np.count_nonzero(np.isnan(col))
plt.figure(figsize=(10, 5))
plt.hist(nb_nans, bins=20)
plt.title("Number of NaNs per feature after")

Standardize and remove nans

In [ ]:
# remove the columns that have the same value in it (avoid std=0)
reduced_x_train, reduced_ids = remove_identic_col(reduced_ids, reduced_x_train)
nb_col_reduced = reduced_x_train.shape[1]
print(nb_col_reduced)

In [ ]:
x_standardized = standardize_data(reduced_x_train)

clean_x_train = replace_NaN(x_standardized, method = 'mean')

## Run the methods a first time to see how we're doing

In [ ]:
from implementations import *
NB_COL = clean_x_train.shape[1] # corresponds to 'D' = number of features
NB_ROWS = clean_x_train.shape[0] # corresponds to 'N' = number of observations/respondents

### Gradient descent

In [ ]:
w_mse_gd, loss_mse_gd = mean_squared_error_gd(y_train, clean_x_train, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.001)
print(loss_mse_gd)
y_mse_gd = clean_x_train @ w_mse_gd
y_mse_gd = convert_predict(y_mse_gd)
print(y_mse_gd)

In [ ]:
print(y_train)

### Subgradient descent

In [ ]:
w_mse_sgd, loss_mse_sgd = mean_squared_error_sgd(y_train, clean_x_train, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.001)
print(loss_mse_gd)
y_mse_gd = clean_x_train @ w_mse_gd
y_mse_gd = convert_predict(y_mse_gd)
print(y_mse_gd)

In [ ]:
w_ls, loss_ls = least_squares(y_train, x_train)
w_rr, loss_rr = ridge_regression(y_train, x_train, lambda_ = 0.5)
w_lrgd, loss_lrgd = logistic_regression(y_train, x_train, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.1, lambda_ = 0.5)
w_lrsgd, loss_lrsgd = logistic_regression_SGD(y_train, x_train, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.1, lambda_ = 0.5)
w_reg_lrgd, loss_reg_lrgd = reg_logistic_regression(y_train, x_train, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.1, lambda_ = 0.5)
w_reg_lrsgd, loss_reg_lrsgd = reg_logistic_regression_SGD(y_train, x_train, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.1, lambda_ = 0.5)

Idea to remove columns: compare means in population y=0 and y=1, and remove col for which those means are close
(this is done already a little with remove_identic_col for the columns that have the same value in each cell)

Cleaning outliers

In [ ]:
removed_outlier_x_train = clean_outliers(reduced_x_train)

In [ ]:
features_below_10_unique_values = {}
for i in range(reduced_x_train.shape[1]):
    # print(f"{reduced_features[i]}: {len(np.unique(reduced_data[:,i]))}")
    #print(f"{reduced_features[i]}: {np.unique(reduced_x_train[:,i])}")
    if len(np.unique(reduced_x_train[:,i])) < 10:
        features_below_10_unique_values[reduced_features[i]] = len(np.unique(reduced_x_train[:,i]))

# dealing with correlation 

def create_dictionary_from_correlation(correlation_threshold):
    newfeature_correlation_dict = {}
    # For each feature in the dataset calculate the correlation with the others and save those which have higher than 0.6 correlation
    for ft_num, feature in enumerate(reduced_features):
        newfeature_correlation_dict[feature] = []
        for o_ft_num, other_feature in enumerate(reduced_features):
            if (feature != other_feature):
                if np.abs(np.corrcoef(reduced_x_train[:,ft_num],reduced_x_train[:,o_ft_num])[0,1]) >= correlation_threshold:
                    newfeature_correlation_dict[feature].append(other_feature)
        print(f" Finished for feature: {feature}")
    return newfeature_correlation_dict

feature_correlation_dict = create_dictionary_from_correlation(0.6)
with open('feature_correlation_dict.pickle', 'wb') as handle:
    pickle.dump(feature_correlation_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('feature_correlation_dict.pickle', 'rb') as handle:
    feature_correlation_dict = pickle.load(handle)

### Run the functions on the data

In [ ]:
from implementations import *
w_mse_gd, loss_mse_gd = mean_squared_error_gd(y_train, x_train, initial_w = np.array([0,0]), max_iters = 50, gamma = 0.1)
w_mse_sgd, loss_mse_sgd = mean_squared_error_sgd(y_train, x_train, initial_w = np.array([0,0]), max_iters = 50, gamma = 0.1)
w_ls, loss_ls = least_squares(y_train, x_train)
w_rr, loss_rr = ridge_regression(y_train, x_train, lambda_ = 0.5)
w_lrgd, loss_lrgd = logistic_regression_GD(y_train, x_train, initial_w = np.array([0,0]), max_iters = 50, gamma = 0.1, lambda_ = 0.5)
w_lrsgd, loss_lrsgd = logistic_regression_SGD(y_train, x_train, initial_w = np.array([0,0]), max_iters = 50, gamma = 0.1, lambda_ = 0.5)
w_reg_lrgd, loss_reg_lrgd = regularized_logistic_regression_GD(y_train, x_train, initial_w = np.array([0,0]), max_iters = 50, gamma = 0.1, lambda_ = 0.5)
w_reg_lrsgd, loss_reg_lrsgd = regularized_logistic_regression_SGD(y_train, x_train, initial_w = np.array([0,0]), max_iters = 50, gamma = 0.1, lambda_ = 0.5)